##### Importing libs

In [7]:
import cv2
import numpy as np
import mediapipe as mp

from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

##### Run detector

In [13]:
cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volbar=400
volper=0

volMin,volMax = volume.GetVolumeRange()[:2]
 
while True:
    success,img = cap.read()
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    results = hands.process(imgRGB)
 
    lmList = []
    if results.multi_hand_landmarks:
        for handlandmark in results.multi_hand_landmarks:
            for id,lm in enumerate(handlandmark.landmark):
                h,w,_ = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy])
            mpDraw.draw_landmarks(img,handlandmark,mpHands.HAND_CONNECTIONS)
    
    if lmList != []:
        #getting the value at a point
                        #x      #y
        x1,y1 = lmList[4][1],lmList[4][2]
        x2,y2 = lmList[8][1],lmList[8][2]
        #creating circle at the tips of thumb and index finger
        cv2.circle(img,(x1,y1),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.circle(img,(x2,y2),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3)  #create a line b/w tips of index finger and thumb

        length = hypot(x2-x1,y2-y1)*2
        vol = np.interp(length,[30,350],[volMin,volMax]) 
        volbar=np.interp(length,[30,350],[400,150])
        volper=np.interp(length,[30,350],[0,100])

        print(vol, int(length))
        volume.SetMasterVolumeLevel(vol, None)
        
        # Hand range 30 - 350
        # Volume range -63.5 - 0.0
        #creating volume bar for volume level 
        cv2.rectangle(img,(50,150),(85,400),(0,0,255),4) # vid ,initial position ,ending position ,rgb ,thickness
        cv2.rectangle(img,(50,int(volbar)),(85,400),(0,0,255),cv2.FILLED)
        cv2.putText(img,f"{int(volper)}%",(10,40),cv2.FONT_ITALIC,1,(255, 0, 98),3)

    cv2.imshow('Image',img) #Show the video 
    if cv2.waitKey(1) & 0xff==ord(' '): #By using spacebar delay will stop
        break
        
cap.release()
cv2.destroyAllWindows()

-38.19516254843751 62
-37.568232785651894 66
-26.53860691652389 139
-21.93654952867278 170
-21.406897463122558 173
-19.831606479594807 184
-31.65670488818693 105
-31.93465387471258 103
-35.130683123147016 82
-36.207081865168774 75
-34.06683209365713 89
-33.923549801218286 90
-33.923549801218286 90
-39.427825571755776 53
-36.85951128941908 70
-41.344514641395044 41
-40.0 50
-33.906987088948966 90
-29.78983342822547 118
-29.600279331788443 119
-28.030023112494458 129
-26.60741758422382 139
-2.9017937580444766 297
-1.6667151515408065 305
-1.6088243776649378 305
-43.0 15
-43.0 10
-43.0 16
-43.0 10
-15.260117866220416 214
-43.0 19
-43.0 25
-43.0 24
-43.0 26
-25.495455014929302 146
-25.010002223210424 149
-23.777226131836947 158
-24.462422002302414 153
-23.52814149883243 159
-20.423441873088816 180
-12.724577644549022 231
-3.1963884090698613 295
-2.607127514492909 299
-2.607127514492909 299
-2.9017937580444766 297
-2.9017937580444766 297
-2.197019965569595 302
-2.7506983294711134 298
-3.6250